In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /Users/sr7037/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sr7037/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sr7037/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def sort_movies_by_year(li):
    def merge_sort(a,l,r):
        if l==r:
            return
        mid=(l+r)//2
        merge_sort(a,l,mid)
        merge_sort(a,mid+1,r)
        merge(a,l,mid,r)

    def merge(a,l,mid,r):
        n1=mid-l+1
        n2=r-(mid+1)+1
        L=[a[i+l] for i in range(n1)]
        R=[a[i+mid+1] for i in range(n2)]
        i,j,k=0,0,l
        while(i<n1 and j<n2):
            if int(L[i][-5:-1])>int(R[j][-5:-1]) :
                a[k]=L[i]
                i+=1
            else:
                a[k]=R[j]
                j+=1
            k+=1
        while(i<n1):
            a[k]=L[i]
            i+=1
            k+=1
        while(j<n2):
            a[k]=R[j]
            j+=1
            k+=1
    merge_sort(li,0,len(li)-1)

In [3]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
dataset = movies.merge(ratings)
dataset = dataset.loc[:,["userId","movieId","title","genres","rating"]]
df_ratings = dataset.loc[:,["title","rating"]].groupby("title").mean()
genres = dataset["genres"]

In [4]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
li = []
for i in range(len(genres)):
    temp = genres[i].split("|")
    for j in range(len(temp)):
        temp[j] = lemmatizer.lemmatize(temp[j])
    li.append(" ".join(temp))

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X = cv.fit_transform(li).toarray()

genres = pd.DataFrame(X,columns=cv.get_feature_names())
dataset = dataset.iloc[:,:-2]
new_dataset = dataset.join(genres)

In [6]:
new_dataset

,userId,movieId,title,action,adventure,animation,children,comedy,crime,documentary,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
0,2,1,Toy Story (1995),0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,1,Toy Story (1995),0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,1,Toy Story (1995),0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11,1,Toy Story (1995),0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14,1,Toy Story (1995),0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105334,475,148238,A Very Murray Christmas (2015),0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
105335,458,148626,The Big Short (2015),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105336,576,148626,The Big Short (2015),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105337,668,148626,The Big Short (2015),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
users = new_dataset.drop(["movieId","title"],axis=1)
users_moviemat = users.groupby("userId").sum()
X = users_moviemat.iloc[:,:].values
users_moviemat

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,fi,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
userId,,,,,,,,,,,,,,,,,,,,,
1,46,31,2,3,31,31,1,45,8,25,...,0,1,13,0,2,16,25,43,10,1
2,9,10,2,3,11,3,0,11,4,5,...,0,2,2,0,0,8,5,12,0,0
3,13,9,2,5,35,12,1,36,5,3,...,0,3,4,0,0,22,3,21,3,3
4,14,17,4,6,46,18,0,76,8,3,...,0,6,10,0,6,37,3,18,16,5
5,17,22,21,21,45,6,0,19,16,6,...,0,11,3,0,0,21,6,11,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,36,30,2,3,26,9,1,16,8,22,...,0,6,5,0,0,20,22,22,7,0
665,73,51,7,16,60,61,0,128,25,25,...,0,5,24,0,0,31,25,102,17,5
666,34,27,10,19,83,36,1,101,19,30,...,0,2,20,0,7,26,30,51,8,4


In [7]:
from sklearn.neighbors import NearestNeighbors

classifier = NearestNeighbors()
classifier.fit(X)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [10]:
uid = int(input("Enter User Id "))
li = classifier.kneighbors([X[uid-1]],n_neighbors=5,return_distance=False)
current_user = new_dataset.loc[new_dataset["userId"]==li[0][0],:]["title"].values
similar_user = new_dataset.loc[new_dataset["userId"]==li[0][1],:]["title"].values
movies_list = [movie for movie in similar_user if movie not in current_user]
sort_movies_by_year(movies_list)
for i in range(len(movies_list)):
    movies_list[i] = (movies_list[i], df_ratings['rating'][df_ratings.index == movies_list[i]].values[0])
print("Recommended Movies are: ")
movies_list

Enter User Id 108
Recommended Movies are: 


[('Harry Potter and the Deathly Hallows: Part 1 (2010)', 3.7580645161290325),
 ('Black Swan (2010)', 3.9655172413793105),
 ('Devil (2010)', 3.0714285714285716),
 ('Easy A (2010)', 3.3461538461538463),
 ('Social Network, The (2010)', 3.875),
 ('Machete (2010)', 3.3636363636363638),
 ('Scott Pilgrim vs. the World (2010)', 4.0),
 ('Expendables, The (2010)', 3.125),
 ('Salt (2010)', 3.3125),
 ('Karate Kid, The (2010)', 3.3333333333333335),
 ('Inception (2010)', 4.189320388349515),
 ('Toy Story 3 (2010)', 4.142857142857143),
 ('Get Him to the Greek (2010)', 3.15),
 ('Iron Man 2 (2010)', 3.6451612903225805),
 ('Death at a Funeral (2010)', 2.75),
 ('Kick-Ass (2010)', 3.857142857142857),
 ('How to Train Your Dragon (2010)', 3.757142857142857),
 ("She's Out of My League (2010)", 3.0),
 ("Valentine's Day (2010)", 3.0),
 ('Avatar (2009)', 3.856060606060606),
 ('Invictus (2009)', 3.5833333333333335),
 ('Blind Side, The  (2009)', 3.9347826086956523),
 ('Ninja Assassin (2009)', 3.2857142857142856),
